In [ ]:
import numpy as np 
import pandas as pd 
import os
import glob
import cv2
import random
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.utils import to_categorical, plot_model
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, Activation
from keras.models import Model, Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.models import load_model
from keras.utils import np_utils
from keras import backend as K

PATH = "../input/cell-images-for-detecting-malaria/cell_images/cell_images/"
print(os.listdir(PATH))

 * https://github.com/keras-team/keras/issues/5400

In [ ]:
# all image path of parasitized
paths_para=glob.glob(os.path.join(PATH,'Parasitized','*.png'))
paths_uninf=glob.glob(os.path.join(PATH,'Uninfected','*.png'))
len(paths_para), len(paths_uninf)

In [ ]:
# show one
path=paths_para[0]
img=cv2.imread(path) # cv2 stores image in BGR format
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # transforming to RGB format
img.shape

In [ ]:
# helper to get images
def get_data(path, resize_dim=96):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img_rz=cv2.resize(img,(resize_dim,resize_dim),interpolation=cv2.INTER_AREA)
    return img_rz

## Get the data

In [ ]:
parasite_images=np.array([get_data(path) for path in paths_para])
parasite_images.shape

In [ ]:
uninf_images=np.array([get_data(path) for path in paths_uninf])
uninf_images.shape

## Make labels

In [ ]:
para_label = np.array([1 for _ in range(len(parasite_images))])
uninf_label = np.array([0 for _ in range(len(uninf_images))])

para_label.shape, uninf_label.shape

## Data set initialize

In [ ]:
x_all = np.concatenate((parasite_images, uninf_images), axis=0)
y_all = np.concatenate((para_label, uninf_label), axis=0)
print(x_all.shape, y_all.shape)

#train = shuffle(train, random_state=123)

## Show an image

In [ ]:
idx = random.randint(0, len(x_all))
print("Index number: {}".format(idx))
print("Label: {}".format(y_all[idx]))
plt.imshow(x_all[idx])

## Show many images

In [ ]:
plt.figure(figsize=(20,20))
columns = 5

lower = 0
upper = 30 

for i in range(upper):
    ax = plt.subplot(upper / columns + 1, columns, i + 1)
    idx = random.randint(0, len(x_all))
    ax.set_title(y_all[idx])
    plt.imshow(x_all[idx], cmap='gray') # binary
    
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

## Make training and testing sets

In [ ]:
indices=list(range(len(x_all)))
np.random.seed(42)
np.random.shuffle(indices)

ind=int(len(indices)*0.85)
# train data
input_features=x_all[indices[:ind]] 
labels=y_all[indices[:ind]]
# validation data
x_test=x_all[indices[-(len(indices)-ind):]] 
y_test=y_all[indices[-(len(indices)-ind):]]

In [ ]:
print(input_features.shape, labels.shape)
print(x_test.shape, y_test.shape)

## Define network

In [ ]:
def first():
    input_shape = (96, 96, 3)
    model = Sequential()
    model.add(Conv2D(32, 3, 3, border_mode='same', input_shape=input_shape, activation='relu'))
    model.add(Conv2D(32, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(Conv2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, 3, 3, border_mode='same', activation='relu'))
    model.add(Conv2D(256, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy',
                optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0), #RMSprop(lr=0.0001) #deafult LR=0.001
                metrics=['accuracy'])
                # metrics=[mcor,recall, f1]
    return model

In [ ]:
model=first()
model.summary()

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(input_features, labels, test_size=0.2, random_state=1)
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

In [ ]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          validation_data=(x_val, y_val))

print("Done training...")

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
'''
80-20% split -> 92.42% 5 epoch
80-20% split -> 96.03% 10 epoch
'''

# Transfer Learning

In [ ]:
# model with ALL layers frozen, except for the SOFTMAX layer
def vgg16():
    base_model = VGG16(weights='../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',include_top=False,pooling='avg',input_shape=(96, 96, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    # softmax layer
    predictions = Dense(2, activation='softmax')(X)
    
    # magical line of freezing layers
    for layer in base_model.layers:
        layer.trainable=False

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model=vgg16()
model.summary()

In [ ]:
# check input
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

In [ ]:
# one hot encoded vectors
num_classes = 2

y_train = np_utils.to_categorical(y_train,num_classes)
y_val = np_utils.to_categorical(y_val,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)
print(y_train.shape, y_val.shape, y_test.shape)

In [ ]:
# check input
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

In [ ]:
model=vgg16()
path_model='vgg16.h5'  

# set the learning rate
K.set_value(model.optimizer.lr,1e-2) 

h=model.fit(x=x_train,     
            y=y_train, 
            batch_size=64, 
            epochs=50, 
            verbose=1, 
            validation_data=(x_val,y_val),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
            )


In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def plot_loss_accu(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    plt.plot(epochs, loss, 'r')
    plt.plot(epochs, val_loss, 'b')
    plt.title('Training and validation loss')
    plt.legend(['train', 'val'], loc='upper right')
    plt.show()
    loss = history.history['acc']
    val_loss = history.history['val_acc']
    epochs = range(len(loss))
    plt.plot(epochs, loss, 'r')
    plt.plot(epochs, val_loss, 'b')
    plt.title('Training and validation accuracy')
    plt.legend(['train', 'val'], loc='lower right')
    plt.show()


In [ ]:
plot_loss_accu(h)

In [ ]:
def vgg16_scratch():
    base_model = VGG16(weights='../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(10, activation='softmax')(X)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model